# TASK 5

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### Get the LLM

In [ ]:
%%capture
%%bash
pip install -q -U bitsandbytes
pip install -q -U git+https://github.com/huggingface/transformers.git
pip install -q -U git+https://github.com/huggingface/peft.git
pip install -q -U git+https://github.com/huggingface/accelerate.git

In [ ]:
!pip install jsonlines

In [ ]:
import os
import json
import tqdm
import sys
import gzip
from collections import defaultdict

In [ ]:
!pip install torch==2.3.0

In [ ]:
%%capture
from transformers import AutoTokenizer, LlamaForCausalLM, AutoModelForCausalLM

model_name = "HuggingFaceH4/zephyr-7b-beta"
tokenizer = AutoTokenizer.from_pretrained(model_name, truncation=True, padding=True, padding_side="left", maximum_length = 2048, model_max_length = 2048)
model = AutoModelForCausalLM.from_pretrained(model_name, load_in_4bit = True, device_map = 'auto')
tokenizer.pad_token = tokenizer.eos_token
model.generation_config.pad_token_id = model.generation_config.eos_token_id

### Install and import packages

In [ ]:
!pip install pytrec_eval

  Preparing metadata (setup.py) ... done
  Created wheel for pytrec_eval: filename=pytrec_eval-0.5-cp310-cp310-linux_x86_64.whl size=308217 sha256=3c81b7ab37102bcc89e5f95611b7e9400e9cc0b31a4a40ed1cd37413ba407438
  Stored in directory: /root/.cache/pip/wheels/51/3a/cd/dcc1ddfc763987d5cb237165d8ac249aa98a23ab90f67317a8
Successfully built pytrec_eval


In [ ]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 1.6 MB/s eta 0:00:00


In [ ]:
# Imports
import operator
from collections import defaultdict
import tqdm
import jsonlines
import numpy as np
import pytrec_eval
from sentence_transformers.cross_encoder import CrossEncoder
import os
import logging
from sentence_transformers import util
import gzip
import json
import sys

### Get data

In [ ]:
#ranking_run_file_path = "/content/MiniLM"
#ranking_run_file_path = "/content/tinyBERT"
ranking_run_file_path = "/content/gdrive/MyDrive/cross-encoder-reranker-ir-course-2023/finetuned_models/cross-encoder-distilbert-distilroberta-base-2024-05-05_10-29-29ranking.run"
top_docs = defaultdict(list)

# Parse the ranking file
with open(ranking_run_file_path, "r") as f:
    for line in f:
        parts = line.strip().split()
        qid = parts[0]
        did = parts[2]
        score = float(parts[4])
        top_docs[qid].append((did, score))

# Sort documents by score and keep only the top-3 for each query
for qid in top_docs:
    top_docs[qid] = sorted(top_docs[qid], key=operator.itemgetter(1), reverse=True)[:3]

print(top_docs)

defaultdict(<class 'list'>, {'156493': [('1960255', 0.9419941902160645), ('1960260', 0.9321132302284241), ('6139386', 0.8972063064575195)], '1110199': [('8160520', 0.7269127368927002), ('5857921', 0.6710293889045715), ('3272914', 0.6032164692878723)], '1063750': [('4788297', 0.7590005993843079), ('4788299', 0.7250710725784302), ('3327051', 0.6594907641410828)], '130510': [('8612903', 0.9695038795471191), ('8612910', 0.9457370042800903), ('799647', 0.9132503271102905)], '489204': [('1778458', 0.734449028968811), ('385083', 0.7186551690101624), ('7116533', 0.6437966823577881)], '573724': [('5339620', 0.972711443901062), ('5721829', 0.9615448713302612), ('162151', 0.9609178304672241)], '168216': [('1381477', 0.9623517990112305), ('3830857', 0.9601773023605347), ('8731621', 0.9545367360115051)], '1133167': [('6467517', 0.9697607755661011), ('7486600', 0.9695602059364319), ('8160224', 0.9694837927818298)], '527433': [('8617271', 0.9572929739952087), ('5466807', 0.7033931612968445), ('546681

In [ ]:
# Retrieve the queries
from sentence_transformers import util

queries = {}
queries_filepath = os.path.join('/content/msmarco-test2019-queries.tsv.gz')
if not os.path.exists(queries_filepath):
    logging.info("Download "+os.path.basename(queries_filepath))
    util.http_get('https://msmarco.z22.web.core.windows.net/msmarcoranking/msmarco-test2019-queries.tsv.gz', queries_filepath)

with gzip.open(queries_filepath, 'rt', encoding='utf8') as fIn:
    for line in fIn:
        qid, query = line.strip().split("\t")
        queries[qid] = query

  0%|          | 0.00/4.28k [00:00<?, ?B/s]

In [ ]:
#Read which passages are relevant
relevant_docs = defaultdict(lambda: defaultdict(int))
qrels_filepath = os.path.join("/content/qrels.txt")

if not os.path.exists(qrels_filepath):
    logging.info("Download "+os.path.basename(qrels_filepath))
    util.http_get('https://trec.nist.gov/data/deep/2019qrels-pass.txt', qrels_filepath)


with open(qrels_filepath) as fIn:
    for line in fIn:
        qid, _, pid, score = line.strip().split()
        score = int(score)
        if score > 0:
            relevant_docs[qid][pid] = score

# Only use queries that have at least one relevant passage
relevant_qid = []
for qid in queries:
    if len(relevant_docs[qid]) > 0:
        relevant_qid.append(qid)

  0%|          | 0.00/187k [00:00<?, ?B/s]

In [ ]:
passage_filepath = os.path.join("/content/msmarco-passagetest2019-top1000.tsv.gz")

if not os.path.exists(passage_filepath):
    logging.info("Download "+os.path.basename(passage_filepath))
    util.http_get('https://msmarco.z22.web.core.windows.net/msmarcoranking/msmarco-passagetest2019-top1000.tsv.gz', passage_filepath)



passage_cand = {}
document_contents = {}
with gzip.open(passage_filepath, 'rt', encoding='utf8') as fIn:
    for line in fIn:
        qid, pid, query, passage = line.strip().split("\t")
        if qid not in passage_cand:
            passage_cand[qid] = []

        document_contents[pid] = passage
        passage_cand[qid].append([pid, passage])

  0%|          | 0.00/26.6M [00:00<?, ?B/s]

### Construct PRF prompt

In [ ]:
def construct_prf_prompt(query, top_docs_content):
    context = "\n".join(top_docs_content)
    prompt = f"Answer the following query based on the context:\nContext: {context}\nQuery: {query}\nGive the rationale before answering."
    return prompt

### Generate expanded queries using PRF

In [ ]:
def reformulate_query_with_prf(original_query, top_docs_content, model, tokenizer, max_length=512):
    prompt = construct_prf_prompt(original_query, top_docs_content)

    inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True, max_length=max_length).to(model.device)
    outputs = model.generate(**inputs, max_length=max_length, num_beams=5, no_repeat_ngram_size=2, early_stopping=True)

    new_query = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return new_query

reformulated_queries_prf = {}
for qid in tqdm.tqdm(relevant_qid, desc="Processing queries with PRF"):
    query = queries.get(qid, None)
    if query:
        top_docs_content = [document_contents[did] for did, _ in top_docs[qid]]  # Get the content of top-3 documents
        reformulated_queries_prf[qid] = reformulate_query_with_prf(query, top_docs_content, model, tokenizer)
print(reformulated_queries_prf)

Processing queries with PRF:   0%|          | 0/43 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
Processing queries with PRF:  98%|█████████▊| 42/43 [58:11<01:55, 115.35s/it]

### Evaluation

Without PRF

In [ ]:
import tqdm
import numpy as np
import pytrec_eval
from sentence_transformers.cross_encoder import CrossEncoder

#model_save_path = os.path.join("/content/MiniLM")
#model_save_path = os.path.join("/content/tinyBERT")
model_save_path = os.path.join("/content/gdrive/MyDrive/cross-encoder-reranker-ir-course-2023/finetuned_models/cross-encoder-distilbert-distilroberta-base-2024-05-05_10-29-29")


queries_result_list = []
run = {}
model = CrossEncoder(model_save_path, max_length=512)

for qid in tqdm.tqdm(relevant_qid):
    query = queries[qid]

    cand = passage_cand[qid]
    pids = [c[0] for c in cand]
    corpus_sentences = [c[1] for c in cand]

    cross_inp = [[query, sent] for sent in corpus_sentences]

    if model.config.num_labels > 1: #Cross-Encoder that predict more than 1 score, we use the last and apply softmax
        cross_scores = model.predict(cross_inp, apply_softmax=True)[:, 1].tolist()
    else:
        cross_scores = model.predict(cross_inp).tolist()

    cross_scores_sparse = {}
    for idx, pid in enumerate(pids):
        cross_scores_sparse[pid] = cross_scores[idx]

    sparse_scores = cross_scores_sparse
    run[qid] = {}
    for pid in sparse_scores:
        run[qid][pid] = float(sparse_scores[pid])

100%|██████████| 43/43 [00:23<00:00,  1.85it/s]


In [ ]:
evaluator = pytrec_eval.RelevanceEvaluator(relevant_docs, {'ndcg_cut.10', 'recall_100', 'map_cut.1000'})
scores = evaluator.evaluate(run)

print("Evaluation without PRF")
print("Queries:", len(relevant_qid))
print("NDCG@10: {:.2f}".format(np.mean([ele["ndcg_cut_10"] for ele in scores.values()])*100))
print("Recall@100: {:.2f}".format(np.mean([ele["recall_100"] for ele in scores.values()])*100))
print("MAP@1000: {:.2f}".format(np.mean([ele["map_cut_1000"] for ele in scores.values()])*100))

Evaluation without PRF
Queries: 43
NDCG@10: 68.03
Recall@100: 49.29
MAP@1000: 43.39


With PRF

In [ ]:
prompted_queries_result_list = []
prompted_run = {}
model = CrossEncoder(model_save_path, max_length=512)

for qid in tqdm.tqdm(relevant_qid):
    query = reformulated_queries_prf[qid]

    cand = passage_cand[qid]
    pids = [c[0] for c in cand]
    corpus_sentences = [c[1] for c in cand]

    cross_inp = [[query, sent] for sent in corpus_sentences]

    if model.config.num_labels > 1: #Cross-Encoder that predict more than 1 score, we use the last and apply softmax
        cross_scores = model.predict(cross_inp, apply_softmax=True)[:, 1].tolist()
    else:
        cross_scores = model.predict(cross_inp).tolist()

    cross_scores_sparse = {}
    for idx, pid in enumerate(pids):
        cross_scores_sparse[pid] = cross_scores[idx]

    sparse_scores = cross_scores_sparse
    prompted_run[qid] = {}
    for pid in sparse_scores:
        prompted_run[qid][pid] = float(sparse_scores[pid])

100%|██████████| 43/43 [01:17<00:00,  1.79s/it]


In [ ]:
evaluator_prompted = pytrec_eval.RelevanceEvaluator(relevant_docs, {'ndcg_cut.10', 'recall_100', 'map_cut.1000'})
scores_prompted = evaluator_prompted.evaluate(prompted_run)

print("Evaluation with PRF")
print("Queries:", len(reformulated_queries_prf))
print("NDCG@10: {:.2f}".format(np.mean([ele["ndcg_cut_10"] for ele in scores_prompted.values()])*100))
print("Recall@100: {:.2f}".format(np.mean([ele["recall_100"] for ele in scores_prompted.values()])*100))
print("MAP@1000: {:.2f}".format(np.mean([ele["map_cut_1000"] for ele in scores_prompted.values()])*100))

Evaluation with PRF
Queries: 43
NDCG@10: 46.30
Recall@100: 35.49
MAP@1000: 28.20
